<a href="https://colab.research.google.com/github/pnkjjha/BuildNExperiment/blob/main/Structured_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
!pip install pyspark

In [200]:
from pyspark.sql import SparkSession
from pandas import json_normalize
import pyspark.sql.functions as F
import pyspark.sql.types as T
import random
import requests
import json
import os
import pandas as pd
import datetime

In [189]:
spark=SparkSession.builder.appName('test').getOrCreate()

In [155]:
# url = "https://api.carbonintensity.org.uk/intensity"

# while switch:
#   response = requests.get(url)
#   json_normalize(response.json())
#   message = response.json()
#   json_normalize(message['data']).rename(columns={"intensity.forecast": "intensity_forecast",\
#                                                   "intensity.actual": "intensity_actual",\
#                                                   "intensity.percentile": "intensity_percentile",\
#                                                   "intensity.index": "intensity_index"\
#                                                   }).to_csv("intensity_"+str(random.randint(1,10000))+".csv", index=False)


In [190]:
# read lines from file
def flat_json(line):
  # convert line into json
  line_json = json.loads(line)
# json normalize
  flatline = pd.json_normalize(line_json,sep="_")
  return flatline


#Inputfile generation

In [185]:
%rm -rf input
%mkdir input

In [208]:
datetime.datetime.strftime(datetime.datetime.utcnow(),"%Y-%m-%d %H:%M:%S")

'2024-09-09 09:12:10'

In [237]:
s = requests.Session()

i = 0
url = "https://stream.wikimedia.org/v2/stream/recentchange"
with s.get(url, stream=True) as resp:
  for line in resp.iter_lines(1024,decode_unicode=True):
    # Check if line has some bytes in it
    if line:
      # convert line into json
      # get data lines only
      if line[:4] == "data":
          # print(json.loads(line[5:]))
          with open("./input/recentChanges"+str(random.randint(1,10000))+".json","a",encoding="utf-8") as f:
              flat_json_pdf = flat_json(line[5:])
              # flat_json_pdf["meta_dt"] = pd.to_datetime(flat_json_pdf["meta_dt"])
              flat_json_pdf.to_json(f,orient="records",lines=True,index=False)
          i = i + 1
    if i>=15 :
      f.close()
      break

In [192]:
flat_json_pdf["meta_dt"].head()

,meta_dt
0,2024-09-09T08:31:24Z


In [193]:
# design schema here
dSchema = T.StructType([\
   T.StructField("$Schema", T.StringType())\
  ,T.StructField("id", T.IntegerType())\
  ,T.StructField("type", T.StringType())\
  ,T.StructField("namespace", T.IntegerType())\
  ,T.StructField("title", T.StringType())\
  ,T.StructField("title_url", T.StringType())\
  ,T.StructField("comment", T.StringType())\
  ,T.StructField("timestamp", T.IntegerType())\
  ,T.StructField("user", T.StringType())\
  ,T.StructField("bot", T.BooleanType())\
  ,T.StructField("log_id", T.IntegerType())\
  ,T.StructField("log_type", T.StringType())\
  ,T.StructField("log_action", T.StringType())\
  ,T.StructField("log_action_comment", T.StringType())\
  ,T.StructField("server_url", T.StringType())\
  ,T.StructField("server_name", T.StringType())\
  ,T.StructField("server_script_path", T.StringType())\
  ,T.StructField("wiki", T.StringType())\
  ,T.StructField("parsedcomment", T.StringType())\
  ,T.StructField("meta_uri", T.StringType())\
  ,T.StructField("meta_request_id", T.StringType())\
  ,T.StructField("meta_id", T.StringType())\
  ,T.StructField("meta_dt", T.TimestampType())\
  ,T.StructField("meta_domain", T.StringType())\
  ,T.StructField("meta_stream", T.StringType())\
  ,T.StructField("meta_topic", T.StringType())\
  ,T.StructField("meta_partition", T.IntegerType())\
  ,T.StructField("meta_offset", T.IntegerType())\
  ,T.StructField("log_params_userid", T.IntegerType())\
                      ])

# Folder clean up

In [186]:
%rm -f *.json

In [187]:
%rm -rf *check*

In [188]:
%rm -rf output

In [162]:
#static_df= spark.read.load("recentChanges3550.json",format="json",schema=dSchema)

In [194]:
static_df.dtypes

[('$Schema', 'string'),
 ('id', 'int'),
 ('type', 'string'),
 ('namespace', 'int'),
 ('title', 'string'),
 ('title_url', 'string'),
 ('comment', 'string'),
 ('timestamp', 'int'),
 ('user', 'string'),
 ('bot', 'boolean'),
 ('log_id', 'int'),
 ('log_type', 'string'),
 ('log_action', 'string'),
 ('log_action_comment', 'string'),
 ('server_url', 'string'),
 ('server_name', 'string'),
 ('server_script_path', 'string'),
 ('wiki', 'string'),
 ('parsedcomment', 'string'),
 ('meta_uri', 'string'),
 ('meta_request_id', 'string'),
 ('meta_id', 'string'),
 ('meta_dt', 'timestamp'),
 ('meta_domain', 'string'),
 ('meta_stream', 'string'),
 ('meta_topic', 'string'),
 ('meta_partition', 'int'),
 ('meta_offset', 'int'),
 ('log_params_userid', 'int')]

# Structured Streaming

In [196]:
#create Dataframe representing the stream of input lines from connection to localhost:9999
recentChange = spark.readStream.format("json").schema(dSchema).load("input")


In [ ]:
recentChange.isStreaming

In [198]:
rc_df = recentChange.withColumn("meta_dt", recentChange.meta_dt.cast(T.TimestampType()))
rc_df

DataFrame[$Schema: string, id: int, type: string, namespace: int, title: string, title_url: string, comment: string, timestamp: int, user: string, bot: boolean, log_id: int, log_type: string, log_action: string, log_action_comment: string, server_url: string, server_name: string, server_script_path: string, wiki: string, parsedcomment: string, meta_uri: string, meta_request_id: string, meta_id: string, meta_dt: timestamp, meta_domain: string, meta_stream: string, meta_topic: string, meta_partition: int, meta_offset: int, log_params_userid: int]

In [249]:
# Assuming 'recentChange' is your streaming DataFrame
myquery = rc_df\
   .withWatermark("meta_dt", "10 seconds")\
   .groupBy(F.window("meta_dt","10 seconds")\
         , "type")\
  .count()\
  .writeStream\
  .format("csv")\
  .outputMode("append")\
  .option("path", "output")\
  .option("header","true")\
  .option("checkpointLocation", "checkpoint_dir")\
  .start()



In [209]:
myquery.stop()

In [242]:
inmemoryQuery = rc_df\
  .groupBy(F.window("meta_dt","1 minutes"), "type")\
  .count()\
  .writeStream\
  .queryName("myquery")\
  .format("memory")\
  .outputMode("complete")\
  .start()

In [246]:
spark.sql("select * from myquery order by window").show(truncate=False)

+------------------------------------------+----------+-----+
|window                                    |type      |count|
+------------------------------------------+----------+-----+
|{2024-09-09 08:31:00, 2024-09-09 08:32:00}|categorize|1    |
|{2024-09-09 08:31:00, 2024-09-09 08:32:00}|edit      |3    |
|{2024-09-09 10:10:00, 2024-09-09 10:11:00}|edit      |2    |
|{2024-09-09 10:10:00, 2024-09-09 10:11:00}|categorize|2    |
|{2024-09-09 10:12:00, 2024-09-09 10:13:00}|new       |1    |
|{2024-09-09 10:12:00, 2024-09-09 10:13:00}|categorize|1    |
|{2024-09-09 10:12:00, 2024-09-09 10:13:00}|log       |3    |
|{2024-09-09 10:12:00, 2024-09-09 10:13:00}|edit      |10   |
|{2024-09-09 10:34:00, 2024-09-09 10:35:00}|log       |1    |
|{2024-09-09 10:34:00, 2024-09-09 10:35:00}|edit      |4    |
|{2024-09-09 10:34:00, 2024-09-09 10:35:00}|new       |1    |
|{2024-09-09 10:34:00, 2024-09-09 10:35:00}|categorize|9    |
+------------------------------------------+----------+-----+



In [248]:
inmemoryQuery.stop()